# Retrieve features from regional environmental rasters, previously generated to study the Vegetation Type Distribution in Norway.
#### For details, see <a href="https://doi.org/10.1111/avsc.12451">Horvath et al. (2019)</a> and supplementary materials therein.
<br/>
<span style="color:red">Disclaimer: the following steps require a verified account and sufficient storage space for the "Google Earth Engine" cloud computing software. The scripts are included for full transparency and are not required to run the models. Required output data are stored in the '../data' folder.</span>

-----------

In [ ]:
# Uncomment to authenticate GEE, only needs to be done once. 
# WARNING: Might not work directly in the notebook, use 
# "earthengine authenticate" in the terminal (e,g. anaconda prompt) in that case!
# import ee
#!earthengine authenticate

In [ ]:
### Import libraries
import ee
import pandas as pd

# Initialize Google Earth Engine
ee.Initialize()

#### 0. Access personal Google Earth Engine assets
<span style="color:red;">Obs! The Vegetation Type and baseline environmental predictor datasets have usage restrictions and cannot be accessed publicly. This code retrieves them from a private GEE asset.</span>

In [ ]:
### Store user credentials from Command line tool in variable
project_credentials = !earthengine ls

In [ ]:
### Extract user name
user_name = project_credentials.s.split("/")[-1]

In [ ]:
### Print asset info for VT data
!earthengine asset info users/{user_name}/vtprespoints

In [ ]:
!earthengine asset info users/{user_name}/horetal_categorical

In [ ]:
!earthengine asset info users/{user_name}/horetal_continuous

----------
### 1. Extract values at VT presence points
#### 1.1 Categorical

In [ ]:
import pandas as pd
from src.data import eehelpers as eeh

In [ ]:
### Load categorical data as EE image
hor_cat_img = ee.Image(f'users/{user_name}/horetal_categorical')

In [ ]:
### Initialize VT FeatureCollection
vt_presence_fc = eeh.VTFeatureCollection(
    user_name=user_name,
    vt_asset_name='vtprespoints',  # Name of the asset containing the VT presence points
    extract_columns=[
        'POINT_X',
        'POINT_Y',
        'FLATE_NR',
        'layer'
    ]  # Columns to extract from raw data
)

#### Use the EE servers to extract the values from the categorical predictor raster image at the VT presence point locations
<b style="color:red">Attention!</b> A too low value for "scale" (spatial scale in m or px) may exceed the user memory limit set by EE. Choose a different value for `tile_scale` or `scale`, or find the prepared datasets in the project folder.

From GEE documentation: <br/>
tileScale (Float, default: 1):
A scaling factor used to reduce aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable computations that run out of memory with the default.

In [ ]:
### Extract image data
sampled_horcat_list = vt_presence_fc.extract_from_eeimg(
    img=hor_cat_img,
    scale=100,
    tile_scale=1
)

In [ ]:
### Test if it worked
print(sampled_horcat_list[0])

#### Write to dataframe and csv

In [ ]:
feature_mat_cat = eeh.df_from_sampled_list(
    sampled_list=sampled_horcat_list,
    saveas="baseline_predictors_cat.csv"
)

In [ ]:
### Print head
print(feature_mat_cat.shape)
feature_mat_cat.head()

---
#### 1.1 Continuous

In [ ]:
### Load categorical data as EE image
hor_cont_img = ee.Image(
    f'users/{user_name}/horetal_continuous'
)

In [ ]:
### Initialize VT FeatureCollection
vt_presence_fc = eeh.vt_ee_featcoll(
    user_name=user_name,
    vt_asset_name='vtprespoints',
    extract_columns=[
        'POINT_X',
        'POINT_Y',
        'FLATE_NR',
        'layer'
    ]
)

In [ ]:
### Extract image data
sampled_horcont_list = vt_presence_fc.extract_from_eeimg(
    img=hor_cont_img, 
    scale=100, 
    tileScale=1
)

In [ ]:
### Test if it worked
print(sampled_horcont_list[0])

#### Write to dataframe and csv

In [ ]:
feature_mat_cont = eeh.df_from_sampled_list(
    sampled_list=sampled_horcont_list,
    save_as="baseline_predictors_num.csv"
)

In [ ]:
### Print head
print(feature_mat_cont.shape)
feature_mat_cont.head()

<h2 style="color:salmon;font-weight:bold">The end.</h2>

---
---
---